In [1]:
!pip install boto3 pandas

import os, re, boto3, pandas as pd

MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "http://minio:9000")
MINIO_BUCKET   = os.getenv("MINIO_BUCKET", "demo-bucket")
MINIO_KEY      = os.getenv("MINIO_KEY", "minio-root-user")        # MINIO_ROOT_USER
MINIO_SECRET   = os.getenv("MINIO_SECRET", "minio-root-password")     # MINIO_ROOT_PASSWORD

s3 = boto3.client(
    "s3",
    aws_access_key_id=MINIO_KEY,
    aws_secret_access_key=MINIO_SECRET,
    endpoint_url=MINIO_ENDPOINT,
)

resp = s3.list_objects_v2(
    Bucket=MINIO_BUCKET,
    Prefix="finance/yahoo/daily/",
    Delimiter="/",
)

prefixes = [cp["Prefix"] for cp in resp.get("CommonPrefixes", [])]
latest = sorted([p for p in prefixes if re.search(r"ingest_date=\d{4}-\d{2}-\d{2}/$", p)])[-1]
latest


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 6.3 MB/s eta 0:00:0000:0100:01


'finance/yahoo/daily/ingest_date=2025-08-18/'

In [2]:
!pip install s3fs

storage_options = {
    "key": MINIO_KEY,
    "secret": MINIO_SECRET,
    "client_kwargs": {"endpoint_url": MINIO_ENDPOINT},
}

df = pd.read_parquet(f"s3://{MINIO_BUCKET}/{latest}AAPL.parquet", storage_options=storage_options)
#df.shape, df.head(), 
df.info()
#df.groupby("ticker")["Close"].mean().round(2)
#df.groupby(('ticker',''))[('Close','AAPL')].mean().round(2)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 6.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.40.11
    Uninstalling botocore-1.40.11:
      Successfully uninstalled botocore-1.40.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.40.11 requires botocore<1.41.0,>=1.40.11, but you have botocore 1.39.11 which is incompatible.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ('D

In [3]:
df_cols = list(df.columns)
type(df.columns), [type(c) for c in df_cols], df_cols[:5]


(pandas.core.indexes.base.Index,
 [str, str, str, str, str, str, str, str],
 ["('Date', '')",
  "('Close', 'AAPL')",
  "('High', 'AAPL')",
  "('Low', 'AAPL')",
  "('Open', 'AAPL')"])

In [4]:
import ast
import pandas as pd

# 1) Normalize columns to a real 2-level MultiIndex
def to_two_level_multiindex(df):
    new_cols = []
    for c in df.columns:
        if isinstance(c, tuple) and len(c) == 2:
            new_cols.append(c)
        elif isinstance(c, str) and c.startswith("("):
            try:
                t = ast.literal_eval(c)
                new_cols.append(t if isinstance(t, tuple) and len(t) == 2 else (str(c), ""))
            except Exception:
                new_cols.append((str(c), ""))
        else:
            new_cols.append((str(c), ""))
    out = df.copy()
    out.columns = pd.MultiIndex.from_tuples(new_cols)
    return out

df2 = to_two_level_multiindex(df)

# 2) (Important) Drop or rename the original ('ticker','') to avoid name clash
if ('ticker','') in df2.columns:
    # either drop:
    df2 = df2.drop(columns=[('ticker','')])
    # or rename instead:
    # df2 = df2.rename(columns={( 'ticker',''):('orig_ticker','')})

# 3) Go tidy by stacking level-1 (tickers like 'AAPL')
if ('Date','') in df2.columns:
    df2 = df2.set_index(('Date',''))

long = (
    df2.stack(1)                 # brings level-1 (tickers) into rows
       .reset_index(level=1)     # make that column explicit
       .rename(columns={'level_1': 'ticker'})
       .reset_index()            # bring Date back if you set it
)

# 4) Now grouping works (no duplicate 'ticker')
long.groupby('ticker')['Close'].mean().round(2)


ticker
           NaN
AAPL    216.64
Name: Close, dtype: float64